In [1]:
!pip install pycoingecko

In [22]:
import pandas as pd
from pycoingecko import CoinGeckoAPI
import datetime
import time
import numpy as np

In [3]:
gecko_list = [
    "ethereum"
]

In [4]:
cg = CoinGeckoAPI()

In [5]:
timePeriod = 300

data = {}
for coin in gecko_list:
    try:
        nested_lists = cg.get_coin_market_chart_by_id(
            id=coin, vs_currency="usd", days=timePeriod
        )["prices"]
        data[coin] = {}
        data[coin]["timestamps"], data[coin]["values"] = zip(*nested_lists)

    except Exception as e:
        print(e)
        print("coin: " + coin)

frame_list = [
    pd.DataFrame(data[coin]["values"], index=data[coin]["timestamps"], columns=[coin])
    for coin in gecko_list
    if coin in data
]

In [6]:
df_cyptocurrency = pd.concat(frame_list, axis=1).sort_index()
df_cyptocurrency

,ethereum
1657238400000,1240.378406
1657324800000,1233.514679
1657411200000,1216.849707
1657497600000,1169.012708
1657584000000,1097.449438
...,...
1682812800000,1905.273852
1682899200000,1885.391935
1682985600000,1832.725799
1683072000000,1870.828573


In [7]:
df_cyptocurrency["datetime"] = pd.to_datetime(df_cyptocurrency.index, unit="ms")
df_cyptocurrency["date"] = df_cyptocurrency["datetime"].dt.date
df_cyptocurrency["hour"] = df_cyptocurrency["datetime"].dt.hour
df_cyptocurrency

,ethereum,datetime,date,hour
1657238400000,1240.378406,2022-07-08 00:00:00,2022-07-08,0
1657324800000,1233.514679,2022-07-09 00:00:00,2022-07-09,0
1657411200000,1216.849707,2022-07-10 00:00:00,2022-07-10,0
1657497600000,1169.012708,2022-07-11 00:00:00,2022-07-11,0
1657584000000,1097.449438,2022-07-12 00:00:00,2022-07-12,0
...,...,...,...,...
1682812800000,1905.273852,2023-04-30 00:00:00,2023-04-30,0
1682899200000,1885.391935,2023-05-01 00:00:00,2023-05-01,0
1682985600000,1832.725799,2023-05-02 00:00:00,2023-05-02,0
1683072000000,1870.828573,2023-05-03 00:00:00,2023-05-03,0


In [8]:
df_cyptocurrency = df_cyptocurrency.melt(
    id_vars=["datetime", "date", "hour"], var_name="currency_name", ignore_index=True
)
df_cyptocurrency.head(5)

,datetime,date,hour,currency_name,value
0,2022-07-08,2022-07-08,0,ethereum,1240.378406
1,2022-07-09,2022-07-09,0,ethereum,1233.514679
2,2022-07-10,2022-07-10,0,ethereum,1216.849707
3,2022-07-11,2022-07-11,0,ethereum,1169.012708
4,2022-07-12,2022-07-12,0,ethereum,1097.449438


In [9]:
df_cyptocurrency.dropna(inplace=True)

In [10]:
curr = dict(
    [
        ("ethereum", "ETH"),
    ]
)

df_cyptocurrency["coin_symbol"] = df_cyptocurrency["currency_name"].apply(
    lambda c: curr[c]
)

In [11]:
df_cyptocurrency

,datetime,date,hour,currency_name,value,coin_symbol
0,2022-07-08 00:00:00,2022-07-08,0,ethereum,1240.378406,ETH
1,2022-07-09 00:00:00,2022-07-09,0,ethereum,1233.514679,ETH
2,2022-07-10 00:00:00,2022-07-10,0,ethereum,1216.849707,ETH
3,2022-07-11 00:00:00,2022-07-11,0,ethereum,1169.012708,ETH
4,2022-07-12 00:00:00,2022-07-12,0,ethereum,1097.449438,ETH
...,...,...,...,...,...,...
296,2023-04-30 00:00:00,2023-04-30,0,ethereum,1905.273852,ETH
297,2023-05-01 00:00:00,2023-05-01,0,ethereum,1885.391935,ETH
298,2023-05-02 00:00:00,2023-05-02,0,ethereum,1832.725799,ETH
299,2023-05-03 00:00:00,2023-05-03,0,ethereum,1870.828573,ETH


In [18]:
df_cyptocurrency.dtypes

datetime         datetime64[ns]
date                     object
hour                      int64
currency_name            object
value                   float64
coin_symbol              object
dtype: object

In [30]:
df_cyptocurrency['datetime'] = pd.to_datetime(df_cyptocurrency['datetime'])
df_filtered = df_cyptocurrency[(df_cyptocurrency['datetime'] >= '2022-08-15') & (df_cyptocurrency['datetime'] <= '2022-09-15')]

In [32]:
df_filtered.head(5)

,datetime,date,hour,currency_name,value,coin_symbol
38,2022-08-15,2022-08-15,0,ethereum,1936.701164,ETH
39,2022-08-16,2022-08-16,0,ethereum,1908.277642,ETH
40,2022-08-17,2022-08-17,0,ethereum,1880.600101,ETH
41,2022-08-18,2022-08-18,0,ethereum,1834.826453,ETH
42,2022-08-19,2022-08-19,0,ethereum,1850.111291,ETH


In [33]:
df_filtered.shape

(32, 6)